In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
from sklearn.metrics import mean_absolute_error
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet,BayesianRidge,Ridge,Lasso,LinearRegression
from sklearn.metrics import r2_score
np.seed=0
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True) 
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)

In [4]:
macro_economic_data=pd.read_excel(r"2007 to Jan 2019 Macro Economic Data.xlsx")
macro_economic_data1 = macro_economic_data[(macro_economic_data["DateTime (GMT)"] > '2010-01-01')]
macro_economic_data_AUD=macro_economic_data1[(macro_economic_data1['Currency']=='AUD') ]
macro_economic_data_USD=macro_economic_data1[(macro_economic_data1['Currency']=='USD') ]

In [ ]:
#macro_economic_data_AUD=macro_economic_data_AUD.fillna(0)
#macro_economic_data_USD=macro_economic_data_AUD.fillna(0)

In [3]:
#Load Forex Data

In [5]:
df=pd.read_csv("forex_dataframe_new.csv")
df.columns = df.columns.str.replace('.','_')


In [6]:
df['target1'] = df['bid_close'].shift(-1)
df['target2'] = df['bid_close'].shift(-2)
df['target3'] = df['bid_close'].shift(-3)

In [7]:
df=df[['time_local','target1','target2','target3','bid_close']]

In [8]:
df=df.dropna(how='all')
df['date_local']=pd.DatetimeIndex(df['time_local']).date
df['time_local']=pd.to_datetime(df['time_local'])
df['time_local']=df['time_local'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [9]:
##### Processing macro economic data

In [10]:
macro_economic_data_AUD['DateTime (GMT)']=macro_economic_data_AUD['DateTime (GMT)']+timedelta(hours=4)
macro_economic_data_USD['DateTime (GMT)']=macro_economic_data_USD['DateTime (GMT)']+timedelta(hours=4)

In [11]:
macro_economic_data_USD['Date']=pd.DatetimeIndex(macro_economic_data_USD['DateTime (GMT)']).date
macro_economic_data_AUD['Date']=pd.DatetimeIndex(macro_economic_data_AUD['DateTime (GMT)']).date

In [12]:
def dinominator_val ( priv, act ) :
    if ((act == np.nan) | (act == 0)):
        act = 1
    if ((priv == 0) | (priv == np.nan)):
        priv = act
    return priv

In [13]:
macro_economic_data_USD['PC_macro_economic_ind'] = ( macro_economic_data_USD['Actual'] - macro_economic_data_USD['Previous'] ) /  macro_economic_data_USD[['Previous','Actual']].apply(lambda x: dinominator_val(*x),axis=1) 
macro_economic_data_AUD['PC_macro_economic_ind'] = ( macro_economic_data_AUD['Actual'] - macro_economic_data_AUD['Previous'] ) / macro_economic_data_AUD[['Previous','Actual']].apply(lambda x: dinominator_val(*x), axis=1) 

In [14]:
#macro_economic_data_USD.describe()

In [15]:
#macro_economic_data_AUD['PC_macro_economic_ind'] = ( macro_economic_data_AUD['Actual'] - macro_economic_data_AUD['Previous'] ) / macro_economic_data_AUD['Previous']  
#macro_economic_data_USD['PC_macro_economic_ind'] = ( macro_economic_data_AUD['Actual'] - macro_economic_data_USD['Previous'] ) / macro_economic_data_USD['Previous']  
macro_economic_data_AUD['PC_macro_economic_ind'].fillna(0,inplace=True)
macro_economic_data_USD['PC_macro_economic_ind'].fillna(0,inplace=True)

#### macro_economic_data_summary on Daily Level to be used further on dataset having columns as macroevents

In [16]:
macro_economic_data_USD_summary_vol_pct_daily=macro_economic_data_USD.groupby(['Date'])['Volatility', 'PC_macro_economic_ind' ].mean().reset_index()
macro_economic_data_AUD_summary_vol_pct_daily=macro_economic_data_AUD.groupby(['Date'])['Volatility', 'PC_macro_economic_ind' ].mean().reset_index()

In [17]:
macro_economic_data_USD_summary_vol_pct_daily.rename(index=str, inplace=True, columns={"Volatility": "Volatility_USD_daily", "PC_macro_economic_ind": "PC_macro_economic_ind_USD_daily"})
macro_economic_data_AUD_summary_vol_pct_daily.rename(index=str, inplace=True, columns={"Volatility": "Volatility_AUD_daily", "PC_macro_economic_ind": "PC_macro_economic_ind_AUD_daily"})

#### macro_economic_data_summary on Hourly level to be used further on dataset having columns as macroevents

### Hourly level summary for all macro events converted to columns

In [18]:
macro_economic_data_AUD['hour']=pd.DatetimeIndex(macro_economic_data_AUD['DateTime (GMT)']).hour
macro_economic_data_USD['hour']=pd.DatetimeIndex(macro_economic_data_USD['DateTime (GMT)']).hour

In [19]:
def hour_extract(hour):
        if(hour<=2):
            return 2
        elif (hour>2 and hour<=6):
            return 6
        elif (hour>6 and hour<=10):
            return 10
        elif (hour>10 and hour<=14):
            return 14
        elif (hour>14 and hour<=18):
            return 18
        elif (hour>18 and hour<=22):
            return 22
        else:
            return 2
            

macro_economic_data_AUD['H4_hour']=macro_economic_data_AUD['hour'].apply(hour_extract)
macro_economic_data_USD['H4_hour']=macro_economic_data_USD['hour'].apply(hour_extract)


In [20]:
macro_mean_AUD_Summary=macro_economic_data_AUD.groupby(['Date','H4_hour','Name'])['PC_macro_economic_ind','Volatility'].mean().reset_index()
macro_mean_USD_Summary=macro_economic_data_USD.groupby(['Date','H4_hour','Name'])['PC_macro_economic_ind','Volatility'].mean().reset_index()

In [21]:
macro_mean_AUD_Summary.fillna(0,inplace=True)
macro_mean_AUD_Summary['Datetime']=pd.to_datetime(macro_mean_AUD_Summary.Date) + macro_mean_AUD_Summary.H4_hour.astype('timedelta64[h]')
macro_mean_AUD_Summary['Datetime'] = pd.DatetimeIndex(macro_mean_AUD_Summary['Datetime'])

In [22]:
macro_mean_USD_Summary.fillna(0,inplace=True)
macro_mean_USD_Summary['Datetime']=pd.to_datetime(macro_mean_USD_Summary.Date) + macro_mean_USD_Summary.H4_hour.astype('timedelta64[h]')
macro_mean_USD_Summary['Datetime'] = pd.DatetimeIndex(macro_mean_USD_Summary['Datetime'])

In [23]:
####### Processing volatility 4 hour window

In [24]:
macro_economic_data_AUD_summary_vol_pct_hourly=macro_mean_AUD_Summary.groupby(['Datetime'])['Volatility', 'PC_macro_economic_ind' ].mean().reset_index()
macro_economic_data_USD_summary_vol_pct_hourly=macro_mean_USD_Summary.groupby(['Datetime'])['Volatility', 'PC_macro_economic_ind' ].mean().reset_index()

In [25]:
macro_economic_data_USD_summary_vol_pct_hourly.rename(index=str, inplace=True, columns={"Volatility": "Volatility_USD_hourly", "PC_macro_economic_ind": "PC_macro_economic_ind_USD_hourly"})
macro_economic_data_AUD_summary_vol_pct_hourly.rename(index=str, inplace=True, columns={"Volatility": "Volatility_AUD_hourly", "PC_macro_economic_ind": "PC_macro_economic_ind_AUD_hourly"})

In [26]:
df1=macro_mean_AUD_Summary.pivot(index='Datetime', columns='Name', values='PC_macro_economic_ind').reset_index()
df1.columns = [str(col) + '_PC_AUD_hourly' for col in df1.columns]
df1.rename(columns={'Datetime_PC_AUD_hourly':'Datetime'}, inplace=True)

#df1=df1.fillna(method='ffill',limit=2)
#df1=df1.fillna(method='bfill',limit=2)

In [27]:
df2=macro_mean_AUD_Summary.pivot(index='Datetime', columns='Name', values='Volatility').reset_index()
df2.columns = [str(col) + '_VOL_AUD_hourly' for col in df2.columns]
df2.rename(columns={'Datetime_VOL_AUD_hourly':'Datetime'}, inplace=True)
#df2=df2.fillna(method='ffill',limit=2)
#df2=df2.fillna(method='bfill',limit=2)


In [28]:
macro_mean_AUD_Summary_new_4hourly=pd.merge(df1,df2, how='inner', on='Datetime')


In [29]:
macro_mean_AUD_Summary_new_4hourly.shape

(2262, 175)

In [30]:
df11=macro_mean_USD_Summary.pivot(index='Datetime', columns='Name', values='PC_macro_economic_ind').reset_index()
df11.columns = [str(col) + '_PC_USD_hourly' for col in df11.columns]
df11.rename(columns={'Datetime_PC_USD_hourly':'Datetime'}, inplace=True)
#df11=df11.fillna(method=DateTime (GMT)=2)
#df11=df11.fillna(method='bfill',limit=2)

In [31]:
df22=macro_mean_USD_Summary.pivot(index='Datetime', columns='Name', values='Volatility').reset_index()
df22.columns = [str(col) + '_VOL_USD_hourly' for col in df22.columns]
df22.rename(columns={'Datetime_VOL_USD_hourly':'Datetime'}, inplace=True)
#df22=df22.fillna(method='ffill',limit=2)
#df22=df22.fillna(method='bfill',limit=2)


In [32]:
macro_mean_USD_Summary_new_4hourly=pd.merge(df11,df22, how='inner', on='Datetime')


In [33]:
macro_mean_USD_Summary_new_4hourly.describe()

10-Year Note Auction_PC_USD_hourly  2-Year Note Auction_PC_USD_hourly  \
count                           82.000000                          83.000000   
mean                             0.009638                           0.035610   
std                              0.082646                           0.141752   
min                             -0.172249                          -0.297125   
25%                             -0.047005                          -0.043019   
50%                             -0.004261                           0.025203   
75%                              0.063939                           0.108500   
max                              0.230198                           0.519435   

       3-Month Bill Auction_PC_USD_hourly  3-Year Note Auction_PC_USD_hourly  \
count                          343.000000                          80.000000   
mean                             0.079898                           0.032796   
std                              0.757834                           0.150897   
min                             -1.000000                          -0.281090   
25%                             -0.059028                          -0.055007   
50%                              0.000000                           0.023480   
75%                              0.050000                           0.097987   
max                             12.500000                           0.641243   

       30-Year Bond Auction_PC_USD_hourly  4-Week Bill Auction_PC_USD_hourly  \
count                           85.000000                         342.000000   
mean                             0.001007                           0.096965   
std                              0.060106                           0.696401   
min                             -0.146770                          -1.000000   
25%                             -0.040191                          -0.074432   
50%                             -0.000620                           0.000000   
75%                              0.034722                           0.086646   
max                              0.174899                           7.000000   

       5-Year Note Auction_PC_USD_hourly  52-Week Bill auction_PC_USD_hourly  \
count                          81.000000                           89.000000   
mean                            0.019126                            0.049023   
std                             0.127851                            0.213496   
min                            -0.223404                           -0.534091   
25%                            -0.049865                           -0.045455   
50%                             0.000000                            0.033613   
75%                             0.090282                            0.103825   
max                             0.471831                            1.439024   

       6-Month Bill Auction_PC_USD_hourly  7-Year Note Auction_PC_USD_hourly  \
count                          346.000000                          76.000000   
mean                             0.020817                           0.017802   
std                              0.170763                           0.107068   
min                             -0.557692                          -0.251765   
25%                             -0.035714                          -0.059111   
50%                              0.002039                           0.014404   
75%                              0.051974                           0.072894   
max                              1.500000                           0.339988   

       ADP Employment Change_PC_USD_hourly  \
count                           109.000000   
mean                             -0.338444   
std                               4.064427   
min                             -42.000000   
25%                              -0.183486   
50%                              -0.008734   
75%                               0.189349   
max                               

In [34]:
macro_mean_USD_Summary_new_4hourly['Date']=pd.DatetimeIndex(macro_mean_USD_Summary_new_4hourly['Datetime']).date
macro_mean_USD_Summary_new_4hourly['Datetime']=pd.DatetimeIndex(macro_mean_USD_Summary_new_4hourly['Datetime'])


### Daily level summary for all macro events converted to columns

In [35]:
macro_mean_AUD_Summary_daily=macro_economic_data_AUD.groupby(['Date','Name'])['PC_macro_economic_ind','Volatility'].mean().reset_index()
macro_mean_USD_Summary_daily=macro_economic_data_USD.groupby(['Date','Name'])['PC_macro_economic_ind','Volatility'].mean().reset_index()

In [36]:
macro_mean_AUD_Summary_daily.fillna(0,inplace=True)
#macro_mean_AUD_Summary_daily['Datetime']=pd.to_datetime(macro_mean_AUD_Summary_daily.Date) + macro_mean_AUD_Summary_daily.H4_hour.astype('timedelta64[h]')

In [37]:
macro_mean_USD_Summary_daily.fillna(0,inplace=True)
#macro_mean_USD_Summary_daily['Datetime']=pd.to_datetime(macro_mean_USD_Summary_daily.Date) + macro_mean_USD_Summary_daily.H4_hour.astype('timedelta64[h]')

In [38]:
df1_d=macro_mean_AUD_Summary_daily.pivot(index='Date', columns='Name', values='PC_macro_economic_ind').reset_index()
df1_d.columns = [str(col) + '_PC_AUD_Daily' for col in df1_d.columns]
df1_d.rename(columns={'Date_PC_AUD_Daily':'Datetime'}, inplace=True)

#df1=df1.fillna(method='ffill',limit=2)
#df1=df1.fillna(method='bfill',limit=2)

In [39]:
df2_d=macro_mean_AUD_Summary_daily.pivot(index='Date', columns='Name', values='Volatility').reset_index()
df2_d.columns = [str(col) + '_VOL_AUD_Daily' for col in df2_d.columns]
df2_d.rename(columns={'Date_VOL_AUD_Daily':'Datetime'}, inplace=True)
#df2=df2.fillna(method='ffill',limit=2)
#df2=df2.fillna(method='bfill',limit=2)


In [40]:
macro_mean_AUD_Summary_new_daily=pd.merge(df1_d,df2_d, how='inner', on='Datetime')


In [41]:
df11_d=macro_mean_USD_Summary_daily.pivot(index='Date', columns='Name', values='PC_macro_economic_ind').reset_index()
df11_d.columns = [str(col) + '_PC_USD_Daily' for col in df11_d.columns]
df11_d.rename(columns={'Date_PC_USD_Daily':'Datetime'}, inplace=True)
#df11=df11.fillna(method='ffill',limit=2)
#df11=df11.fillna(method='bfill',limit=2)

In [42]:
df22_d=macro_mean_USD_Summary_daily.pivot(index='Date', columns='Name', values='Volatility').reset_index()
df22_d.columns = [str(col) + '_VOL_USD_Daily' for col in df22_d.columns]
df22_d.rename(columns={'Date_VOL_USD_Daily':'Datetime'}, inplace=True)
#df22=df22.fillna(method='ffill',limit=2)
#df22=df22.fillna(method='bfill',limit=2)


In [43]:
macro_mean_USD_Summary_new_daily=pd.merge(df11_d,df22_d, how='inner', on='Datetime')

In [44]:
macro_mean_AUD_Summary_new_daily['Date']=pd.DatetimeIndex(macro_mean_AUD_Summary_new_daily['Datetime']).date

In [45]:
macro_mean_USD_Summary_new_daily['Date']=pd.DatetimeIndex(macro_mean_USD_Summary_new_daily['Datetime']).date

In [46]:
df['time_local']=pd.DatetimeIndex(df['time_local'])
macro_economic_data_USD_summary_vol_pct_hourly['Datetime']=pd.DatetimeIndex(macro_economic_data_USD_summary_vol_pct_hourly['Datetime'])

##### Final merging all the datasets

In [47]:
df_merge1 =  pd.merge(df, macro_economic_data_USD_summary_vol_pct_hourly, how='left', left_on='time_local', right_on = 'Datetime')

In [48]:
df_merge2 = pd.merge(df_merge1, macro_economic_data_USD_summary_vol_pct_daily, how='left', left_on='date_local', right_on = 'Date')

In [49]:
#df_merge1.head()

In [50]:
#df_merge1.head()

In [51]:
df_merge3 =  pd.merge(df_merge2, macro_economic_data_AUD_summary_vol_pct_hourly, how='left', left_on='time_local', right_on = 'Datetime')

In [52]:
df_merge4 = pd.merge(df_merge3, macro_economic_data_AUD_summary_vol_pct_daily, how='left', left_on='date_local', right_on = 'Date')

In [53]:
df_merge4.describe()

target1       target2       target3     bid_close  \
count  15268.000000  15267.000000  15266.000000  15269.000000   
mean       0.875694      0.875692      0.875690      0.875697   
std        0.122117      0.122121      0.122125      0.122114   
min        0.683600      0.683600      0.683600      0.683600   
25%        0.759400      0.759400      0.759400      0.759400   
50%        0.885650      0.885600      0.885600      0.885700   
75%        0.996325      0.996350      0.996375      0.996300   
max        1.105500      1.105500      1.105500      1.105500   

       Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
count            4863.000000                       4863.000000   
mean                1.454018                         -0.202596   
std                 0.532456                          2.289896   
min                 0.000000                        -63.591827   
25%                 1.000000                         -0.320715   
50%                 1.333333                          0.000000   
75%                 2.000000                          0.030829   
max                 3.000000                         57.000000   

       Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
count          13710.000000                     13710.000000   
mean               1.462878                        -0.227946   
std                0.426085                         1.638006   
min                0.000000                       -47.710832   
25%                1.250000                        -0.418070   
50%                1.500000                        -0.066593   
75%                1.727273                         0.071992   
max                3.000000                        14.000000   

       Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
count            2204.000000                       2204.000000   
mean                1.611808                         -2.153650   
std                 0.672563                        124.661225   
min                 0.000000                      -5733.203390   
25%                 1.000000                         -0.522437   
50%                 1.500000                          0.000000   
75%                 2.000000                          0.045498   
max                 3.000000                       1137.995997   

       Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  
count          10458.000000                     10458.000000  
mean               1.582161                        -2.823621  
std                0.628965                       139.327873  
min                0.000000                     -5733.203390  
25%                1.000000                        -0.662080  
50%                1.500000                        -0.061378  
75%                2.000000                         0.067789  
max                3.000000                       948.329997

In [54]:
######### Now we will merge event macro economic summary

In [55]:
df_merge5 = pd.merge(df_merge4, macro_mean_AUD_Summary_new_4hourly, how='left', left_on='time_local', right_on = 'Datetime')

In [56]:
#df_merge1.head()

In [57]:
df_merge6 = pd.merge(df_merge5, macro_mean_USD_Summary_new_4hourly, how='left', left_on='time_local', right_on = 'Datetime')

In [58]:
df_merge7 = pd.merge(df_merge6, macro_mean_AUD_Summary_new_daily, how='left', left_on='date_local', right_on = 'Date')

In [59]:
#df_merge1.head()

In [60]:
df_merge8 = pd.merge(df_merge7, macro_mean_USD_Summary_new_daily, how='left', left_on='date_local', right_on = 'Date')

In [61]:
df_merge8.shape


(15269, 1221)

In [62]:
df_merge8.describe()

target1       target2       target3     bid_close  \
count  15268.000000  15267.000000  15266.000000  15269.000000   
mean       0.875694      0.875692      0.875690      0.875697   
std        0.122117      0.122121      0.122125      0.122114   
min        0.683600      0.683600      0.683600      0.683600   
25%        0.759400      0.759400      0.759400      0.759400   
50%        0.885650      0.885600      0.885600      0.885700   
75%        0.996325      0.996350      0.996375      0.996300   
max        1.105500      1.105500      1.105500      1.105500   

       Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
count            4863.000000                       4863.000000   
mean                1.454018                         -0.202596   
std                 0.532456                          2.289896   
min                 0.000000                        -63.591827   
25%                 1.000000                         -0.320715   
50%                 1.333333                          0.000000   
75%                 2.000000                          0.030829   
max                 3.000000                         57.000000   

       Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
count          13710.000000                     13710.000000   
mean               1.462878                        -0.227946   
std                0.426085                         1.638006   
min                0.000000                       -47.710832   
25%                1.250000                        -0.418070   
50%                1.500000                        -0.066593   
75%                1.727273                         0.071992   
max                3.000000                        14.000000   

       Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
count            2204.000000                       2204.000000   
mean                1.611808                         -2.153650   
std                 0.672563                        124.661225   
min                 0.000000                      -5733.203390   
25%                 1.000000                         -0.522437   
50%                 1.500000                          0.000000   
75%                 2.000000                          0.045498   
max                 3.000000                       1137.995997   

       Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  \
count          10458.000000                     10458.000000   
mean               1.582161                        -2.823621   
std                0.628965                       139.327873   
min                0.000000                     -5733.203390   
25%                1.000000                        -0.662080   
50%                1.500000                        -0.061378   
75%                2.000000                         0.067789   
max                3.000000                       948.329997   

       ANZ Job Advertisements_PC_AUD_hourly  ANZAC Day_PC_AUD_hourly  \
count                            109.000000                      7.0   
mean                              -0.703494                      0.0   
std                                3.190116                      0.0   
min                              -13.500000                      0.0   
25%                               -1.428571                      0.0   
50%                               -0.750000                      0.0   
75%                                0.333333                      0.0   
max                               15.250000                      0.0   

       AiG Performance of Construction Index_PC_AUD_hourly  \
count                                         107.000000     
mean                                            0.000472     
std                                             0.077394     
min                                            -0.149813     
25%                                            -0.053368     
50%                                            -0.006494     

In [63]:
#### Will export this raw copy first before backfill and forward fill

In [64]:
df_macro_final1 = df_merge8.copy()

In [65]:
df_macro_final1.to_csv("Macro_economic_data_features_daily_4hour_withoutbackfill.csv",index=False)

In [66]:
#### Backfill and forward fill

In [69]:
df_macro_final2 = df_macro_final1.copy()

In [70]:
df_macro_final2=df_macro_final2.fillna(method='bfill',limit=1)
df_macro_final2=df_macro_final2.fillna(method='ffill',limit=1)

In [71]:
df_macro_final2.fillna(0, inplace=True)

In [72]:
df_macro_final2.to_csv("Macro_economic_data_features_daily_4hour_with_fwnbk_fill1.csv",index=False)

In [73]:
df_macro_final2.shape

(15269, 1221)

In [74]:
##### Now let's create co_relation and evaluate

In [75]:
#df_macro_final2['diff_pc_target1']=(df_macro_final2['target1']-df_macro_final2['bid_close'])/df_macro_final2['bid_close']

In [76]:
corr_df=df_macro_final2.corr()

In [77]:
pd.DataFrame(corr_df.reset_index().to_csv("Macro_economic_data_features_corr_plot.csv"))

Empty DataFrame
Columns: []
Index: []

In [78]:
df_macro_final2.describe()

target1       target2       target3     bid_close  \
count  15269.000000  15269.000000  15269.000000  15269.000000   
mean       0.875684      0.875624      0.875564      0.875697   
std        0.122120      0.122326      0.122530      0.122114   
min        0.683600      0.000000      0.000000      0.683600   
25%        0.759400      0.759300      0.759300      0.759400   
50%        0.885600      0.885600      0.885500      0.885700   
75%        0.996300      0.996300      0.996300      0.996300   
max        1.105500      1.105500      1.105500      1.105500   

       Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
count           15269.000000                      15269.000000   
mean                0.928366                         -0.128877   
std                 0.827264                          1.771010   
min                 0.000000                        -63.591827   
25%                 0.000000                         -0.054577   
50%                 1.000000                          0.000000   
75%                 1.571429                          0.000000   
max                 3.000000                         57.000000   

       Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
count          15269.000000                     15269.000000   
mean               1.391101                        -0.210179   
std                0.524557                         1.576937   
min                0.000000                       -47.710832   
25%                1.200000                        -0.373272   
50%                1.444444                        -0.041421   
75%                1.714286                         0.054138   
max                3.000000                        14.000000   

       Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
count           15269.000000                      15269.000000   
mean                0.605022                         -1.002118   
std                 0.878269                         81.503174   
min                 0.000000                      -5733.203390   
25%                 0.000000                          0.000000   
50%                 0.000000                          0.000000   
75%                 1.000000                          0.000000   
max                 3.000000                       1137.995997   

       Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  \
count          15269.000000                     15269.000000   
mean               1.192034                        -2.250052   
std                0.870368                       124.548698   
min                0.000000                     -5733.203390   
25%                0.000000                        -0.408312   
50%                1.250000                         0.000000   
75%                2.000000                         0.000000   
max                3.000000                       948.329997   

       ANZ Job Advertisements_PC_AUD_hourly  ANZAC Day_PC_AUD_hourly  \
count                          15269.000000                  15269.0   
mean                              -0.015066                      0.0   
std                                0.475746                      0.0   
min                              -13.500000                      0.0   
25%                                0.000000                      0.0   
50%                                0.000000                      0.0   
75%                                0.000000                      0.0   
max                               15.250000                      0.0   

       AiG Performance of Construction Index_PC_AUD_hourly  \
count                                       15269.000000     
mean                                            0.000010     
std                                             0.011170     
min                                            -0.149813     
25%                                             0.000000     
50%                                             0.000000     

In [79]:
#df_macro_final2['diff_pc_target1']

In [80]:
#corr_df[((corr_df['target1'] >= 0.05) | (corr_df['target1'] <= -0.05))].shape

In [81]:
#list(corr_df[((corr_df['target1']>= 0.05) | (corr_df['target1'] <= -0.05))].index)

In [82]:
##### Now let's create co_relation and evaluate

In [83]:
df_macro_final2.head()

time_local  target1  target2  target3  bid_close  date_local  \
0 2010-01-05 18:00:00   0.9117   0.9135   0.9142     0.9124  2010-01-05   
1 2010-01-05 22:00:00   0.9135   0.9142   0.9142     0.9117  2010-01-05   
2 2010-01-06 02:00:00   0.9142   0.9142   0.9156     0.9135  2010-01-06   
3 2010-01-06 06:00:00   0.9142   0.9156   0.9184     0.9142  2010-01-06   
4 2010-01-06 10:00:00   0.9156   0.9184   0.9196     0.9142  2010-01-06   

            Datetime_x  Volatility_USD_hourly  \
0  2010-01-05 22:00:00                    2.0   
1  2010-01-05 22:00:00                    2.0   
2  2010-01-06 02:00:00                    3.0   
3  2010-01-06 02:00:00                    3.0   
4                    0                    0.0   

   PC_macro_economic_ind_USD_hourly      Date_x  Volatility_USD_daily  \
0                         -2.477564  2010-01-05                   2.0   
1                         -2.477564  2010-01-05                   2.0   
2                          0.000000  2010-01-06                   1.5   
3                          0.000000  2010-01-06                   1.5   
4                          0.000000  2010-01-06                   1.5   

   PC_macro_economic_ind_USD_daily           Datetime_y  \
0                        -2.477564                    0   
1                        -2.477564                    0   
2                        -0.395704  2010-01-06 06:00:00   
3                        -0.395704  2010-01-06 06:00:00   
4                        -0.395704  2010-01-06 06:00:00   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly      Date_y  \
0                    0.0                          0.000000  2010-01-05   
1                    0.0                          0.000000  2010-01-05   
2                    0.5                         -1.543434  2010-01-06   
3                    0.5                         -1.543434  2010-01-06   
4                    0.5                         -1.543434  2010-01-06   

   Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily           Datetime_x  \
0                   1.0                        -1.050000                    0   
1                   1.0                        -1.050000                    0   
2                   0.5                        -1.543434  2010-01-06 06:00:00   
3                   0.5                        -1.543434  2010-01-06 06:00:00   
4                   0.5                        -1.543434  2010-01-06 06:00:00   

   ANZ Job Advertisements_PC_AUD_hourly  ANZAC Day_PC_AUD_hourly  \
0                                   0.0                      0.0   
1                                   0.0                      0.0   
2                                   0.0                      0.0   
3                                   0.0                      0.0   
4                                   0.0                      0.0   

   AiG Performance of Construction Index_PC_AUD_hourly  \
0                                                0.0     
1                                                0.0     
2                                                0.0     
3                                                0.0     
4                                                0.0     

   AiG Performance of Mfg Index_PC_AUD_hourly  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   AiG Performance of Services Index_PC_AUD_hourly  \
0                                              0.0   
1                                              0.0   
2                                              0.0   
3                                              0.0   
4                                              0.0   

   Australia Day_PC_AUD_hourly  Australia Day (Observed)_PC_AUD_hourly  \
0                          0.0                                     0.0   
1            

In [84]:
li=df_macro_final2[df_macro_final2.columns[1:]].corr()['target1'][:-1].to_frame().reset_index()
li.dropna(axis=0,inplace=True)
cols_new=li.loc[((li['target1'] >= 0.1) | (li['target1'] <= -0.1))]['index'].tolist()
df_macro_final3=df_macro_final2[cols_new]
#df_macro_final3=df_macro_final2.copy()

In [85]:
df_macro_final3.shape

(15269, 56)

In [86]:
df_macro_final3.describe()

target1       target2       target3     bid_close  \
count  15269.000000  15269.000000  15269.000000  15269.000000   
mean       0.875684      0.875624      0.875564      0.875697   
std        0.122120      0.122326      0.122530      0.122114   
min        0.683600      0.000000      0.000000      0.683600   
25%        0.759400      0.759300      0.759300      0.759400   
50%        0.885600      0.885600      0.885500      0.885700   
75%        0.996300      0.996300      0.996300      0.996300   
max        1.105500      1.105500      1.105500      1.105500   

       Volatility_USD_hourly  CFTC AUD NC net positions_VOL_AUD_hourly  \
count           15269.000000                              15269.000000   
mean                0.928366                                  0.026524   
std                 0.827264                                  0.160694   
min                 0.000000                                  0.000000   
25%                 0.000000                                  0.000000   
50%                 1.000000                                  0.000000   
75%                 1.571429                                  0.000000   
max                 3.000000                                  1.000000   

       API Weekly Crude Oil Stock_VOL_USD_hourly  \
count                               15269.000000   
mean                                    0.037331   
std                                     0.196698   
min                                     0.000000   
25%                                     0.000000   
50%                                     0.000000   
75%                                     0.000000   
max                                     2.000000   

       Baker Hughes US Oil Rig Count_VOL_USD_hourly  \
count                                  15269.000000   
mean                                       0.068570   
std                                        0.358767   
min                                        0.000000   
25%                                        0.000000   
50%                                        0.000000   
75%                                        0.000000   
max                                        2.000000   

       CFTC Gold NC net positions_VOL_USD_hourly  \
count                               15269.000000   
mean                                    0.026328   
std                                     0.160114   
min                                     0.000000   
25%                                     0.000000   
50%                                     0.000000   
75%                                     0.000000   
max                                     1.000000   

       CFTC Oil NC net positions_VOL_USD_hourly  \
count                              15269.000000   
mean                                   0.026328   
std                                    0.160114   
min                                    0.000000   
25%                                    0.000000   
50%                                    0.000000   
75%                                    0.000000   
max                                    1.000000   

       CFTC USD NC net positions_VOL_USD_hourly  \
count                              15269.000000   
mean                                   0.026524   
std                                    0.160694   
min                                    0.000000   
25%                                    0.000000   
50%                                    0.000000   
75%                                    0.000000   
max                                    1.000000   

       Former Fed's Bernanke speech_VOL_USD_hourly  \
count                                 15269.000000   
mean                                      0.030257   
std                                       0.255925   
min                                       0.000000   
25%                                       0.000000   
50%                                       0.000000   
75%                        

In [87]:
df_macro_final3['time_local']=df_macro_final2['time_local']

In [88]:
df_macro_final3.to_csv("Macro_economic_data_features_daily_4hour_with_0.1_corr.csv",index=False)

In [89]:
li2=df_macro_final2[df_macro_final2.columns[1:]].corr()['target1'][:-1].to_frame().reset_index()
li2.dropna(axis=0,inplace=True)
cols_new2=li2.loc[((li2['target1'] >= 0.01) | (li2['target1'] <= -0.01))]['index'].tolist()
df_macro_final4=df_macro_final2[cols_new2]
#df_macro_final3=df_macro_final2.copy()

In [90]:
df_macro_final4['time_local']=df_macro_final2['time_local']

In [91]:
df_macro_final4.to_csv("Macro_economic_data_features_daily_4hour_with_0.01_corr.csv",index=False)